In [1]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
from tqdm import tqdm
import networkx as nx
import json

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

N=16153

%pwd

'/home/cseadmin/data/cys/TrafficFlowModel/data_process'

geo

---

In [71]:
sz_graph_drive=nx.read_gpickle("../data/sz_graph_drive.pkl")
sz_line_graph_drive=nx.read_gpickle("../data/sz_line_graph_drive.pkl")

list(sz_line_graph_drive.nodes)[:10]

[(598153574, 2767949724, 0),
 (2767949724, 2768006811, 0),
 (2767949724, 2768006811, 1),
 (598153574, 297121337, 0),
 (297121337, 2320506360, 0),
 (297121337, 1824553511, 0),
 (598153603, 598153605, 0),
 (598153605, 598153609, 0),
 (685056781, 6383446647, 0),
 (6383446647, 6383446646, 0)]

In [77]:
df_roads=pd.DataFrame()

df_roads["geo_id"]=sz_line_graph_drive.nodes
df_roads=df_roads.reset_index(drop=True)

df_roads["type"]="LineString"

df_roads["coordinates"]=None

df_roads

,geo_id,type,coordinates
0,"(598153574, 2767949724, 0)",LineString,None
1,"(2767949724, 2768006811, 0)",LineString,None
2,"(2767949724, 2768006811, 1)",LineString,None
3,"(598153574, 297121337, 0)",LineString,None
4,"(297121337, 2320506360, 0)",LineString,None
...,...,...,...
84301,"(9148694973, 9148694974, 0)",LineString,None
84302,"(9148694975, 9148694973, 0)",LineString,None
84303,"(9148694975, 9148694973, 1)",LineString,None
84304,"(9148694975, 9148694976, 0)",LineString,None


In [78]:
highway_types=["motorway", "trunk", "primary", "secondary", "motorway_link", "trunk_link", "primary_link", "secondary_link"]
def get_coords(row):
    road=row["geo_id"]

    road_data_dict=sz_graph_drive.get_edge_data(road[0], road[1])[road[2]]

    if isinstance(road_data_dict["highway"], list):
        for road_type in road_data_dict["highway"]:
            if road_type not in highway_types:
                return row
    elif isinstance(road_data_dict["highway"], str):
        if road_data_dict["highway"] not in highway_types:
            return row

    if "geometry" not in road_data_dict:
        return row

    row["coordinates"]=list(road_data_dict["geometry"].coords)

    return row

df_roads=df_roads.apply(get_coords, axis=1)

df_roads

,geo_id,type,coordinates
0,"(598153574, 2767949724, 0)",LineString,"[(114.1742099, 22.5622634), (114.1744957, 22.5..."
1,"(2767949724, 2768006811, 0)",LineString,None
2,"(2767949724, 2768006811, 1)",LineString,"[(114.1782142, 22.5595428), (114.1791269, 22.5..."
3,"(598153574, 297121337, 0)",LineString,"[(114.1742099, 22.5622634), (114.174588, 22.56..."
4,"(297121337, 2320506360, 0)",LineString,"[(114.2259536, 22.5668991), (114.2284345, 22.5..."
...,...,...,...
84301,"(9148694973, 9148694974, 0)",LineString,None
84302,"(9148694975, 9148694973, 0)",LineString,None
84303,"(9148694975, 9148694973, 1)",LineString,None
84304,"(9148694975, 9148694976, 0)",LineString,None


In [79]:
df_roads=df_roads.dropna()
df_roads=df_roads.reset_index(drop=True)
df_roads["geo_id"]=df_roads.index

df_roads

,geo_id,type,coordinates
0,0,LineString,"[(114.1742099, 22.5622634), (114.1744957, 22.5..."
1,1,LineString,"[(114.1782142, 22.5595428), (114.1791269, 22.5..."
2,2,LineString,"[(114.1742099, 22.5622634), (114.174588, 22.56..."
3,3,LineString,"[(114.2259536, 22.5668991), (114.2284345, 22.5..."
4,4,LineString,"[(114.2259536, 22.5668991), (114.2284254, 22.5..."
...,...,...,...
16148,16148,LineString,"[(114.0159237, 22.6483238), (114.015941, 22.64..."
16149,16149,LineString,"[(114.0105371, 22.6496528), (114.0085801, 22.6..."
16150,16150,LineString,"[(114.0122871, 22.6624932), (114.0114325, 22.6..."
16151,16151,LineString,"[(114.0122368, 22.6624536), (114.0122989, 22.6..."


In [80]:
# df_roads.to_csv("../data/libcity_atom/sz_taxi.geo", index=False)

rel

---

In [2]:
tm0_recalc_0diag=np.load("../data/trans_matrix_recalc.npy")[0]

for i in range(len(tm0_recalc_0diag)):
    tm0_recalc_0diag[i][i]=0

In [4]:
rel=[]
rel_id_counter=0

for i in range(N):
    for j in range(N):
        if (tm0_recalc_0diag[i][j]>0):
            rel.append([rel_id_counter, "geo", i, j])
            rel_id_counter+=1

rel_df=pd.DataFrame(rel, columns=["rel_id", "type", "origin_id", "destination_id"])

rel_df

,rel_id,type,origin_id,destination_id,link_weight
0,0,geo,0,1,1
1,1,geo,0,2,1
2,2,geo,0,3,1
3,3,geo,0,4,1
4,4,geo,0,5,1
...,...,...,...,...,...
1798811,1798811,geo,16152,16121,1
1798812,1798812,geo,16152,16124,1
1798813,1798813,geo,16152,16135,1
1798814,1798814,geo,16152,16144,1


In [5]:
# rel_df.to_csv("../data/libcity_atom/sz_taxi.rel", index=False)

dyna

---

1. 给车牌 id，用文件名做 dict
2. 划分轨迹 (按那篇文章)
   * 10min 无点 划分
   * 2min 同一路段 划分 吗?
   * 连续两点之间没路 划分 吗?
3. 转换时间格式 ISO-8601
4. Recover (dij)
5. 时间划分: 15min 现在数据量太小

\[dyna_id, type:"state", time(ISO-8601), entity_id: 车牌 id, traj_id: 同一车的第几条轨迹, location: road_id, properties: 其他 \]

In [2]:
taxi_dict={}
taxi_counter=0

for taxi_file in os.listdir("../data/taxi_after_proc/tracks"):
    taxi_license=taxi_file.split("_")[1].split(".")[0]
    if taxi_license not in taxi_dict:
        taxi_dict[taxi_license]=taxi_counter
        taxi_counter+=1
        
len(taxi_dict)

20868

In [52]:
# dyna_file=open("../data/libcity_atom/sz_taxi_train.dyna", "w")
# write_length=dyna_file.write("dyna_id,type,time,entity_id,traj_id,location,coordinates,speed\n") # disable printing return value

# # df_dyna=pd.DataFrame(columns=["dyna_id", "type", "time", "entity_id", "traj_id", "location", "coordinates", "speed"])
# dyna_id_counter=0

# for taxi_file in tqdm(os.listdir("../data/taxi_after_proc/tracks")):
#     taxi_license=taxi_file.split("_")[1].split(".")[0]
#     df_taxi=pd.read_csv(f"../data/taxi_after_proc/tracks/{taxi_file}", parse_dates=["gps_time"]).dropna() # drop?
#     if df_taxi.empty:
#         continue
#     # df_taxi["gps_time"]=df_taxi["gps_time"].dt.strftime('%Y-%m-%dT%H:%M:%SZ') # ISO-8601
    
#     traj_counter=0
#     last_time=df_taxi.iloc[0]["gps_time"]+pd.Timedelta(seconds=-600)
#     for row in df_taxi.itertuples():
#         if row[3]-last_time>pd.Timedelta(seconds=600):
#             traj_counter+=1
#         last_time=row[3]
        
#         # df_dyna.loc[len(df_dyna)]=[dyna_id_counter, "state", row[3].strftime('%Y-%m-%dT%H:%M:%SZ'), taxi_dict[taxi_license], traj_counter, int(row[8]), [row[2], row[1]], row[4]]
#         write_length=dyna_file.write(f"{dyna_id_counter},state,{row[3].strftime('%Y-%m-%dT%H:%M:%SZ')},{taxi_dict[taxi_license]},{traj_counter},{int(row[8])},\"{[row[2], row[1]]}\",{row[4]}\n")
#         dyna_id_counter+=1
        
# dyna_file.close()

100%|██████████| 123193/123193 [21:35<00:00, 95.09it/s] 


In [66]:
# # Mon. to Fri.
# dyna_file=open("../data/libcity_atom/sz_taxi.dyna", "w")
# write_length=dyna_file.write("dyna_id,type,time,entity_id,traj_id,location,coordinates,speed\n") # disable printing return value

# dyna_id_counter=0

# for taxi_file in tqdm(os.listdir("../data/taxi_after_proc/tracks")):
#     if (int(taxi_file.split("_")[0].split("-")[1])>6):
#         continue
#     taxi_license=taxi_file.split("_")[1].split(".")[0]
#     df_taxi=pd.read_csv(f"../data/taxi_after_proc/tracks/{taxi_file}", parse_dates=["gps_time"]).dropna() # drop?
#     if df_taxi.empty:
#         continue
    
#     traj_counter=0
#     last_time=df_taxi.iloc[0]["gps_time"]+pd.Timedelta(seconds=-600)
#     for row in df_taxi.itertuples():
#         if row[3]-last_time>pd.Timedelta(seconds=600):
#             traj_counter+=1
#         last_time=row[3]
        
#         # df_dyna.loc[len(df_dyna)]=[dyna_id_counter, "state", row[3].strftime('%Y-%m-%dT%H:%M:%SZ'), taxi_dict[taxi_license], traj_counter, int(row[8]), [row[2], row[1]], row[4]]
#         write_length=dyna_file.write(f"{dyna_id_counter},state,{row[3].strftime('%Y-%m-%dT%H:%M:%SZ')},{taxi_dict[taxi_license]},{traj_counter},{int(row[8])},\"{[row[2], row[1]]}\",{row[4]}\n")
#         dyna_id_counter+=1
        
# dyna_file.close()

100%|██████████| 123193/123193 [18:15<00:00, 112.50it/s]


In [8]:
# Mon. to Fri. 12.02-12.06
# 15 min agg interval 24*60/15=96

dyna_file=open("../data/libcity_atom/sz_taxi_3.dyna", "w")
write_length=dyna_file.write("dyna_id,type,time,entity_id,flow\n") # disable printing return value

flow_matrix=np.zeros((5, 96, N), dtype=np.int16)

for taxi_file in tqdm(sorted(os.listdir("../data/taxi_after_proc/tracks"))):
    day=int(taxi_file.split("_")[0].split("-")[1])
    if day>6:
        break
    df_taxi=pd.read_csv(f"../data/taxi_after_proc/tracks/{taxi_file}", parse_dates=["gps_time"]).dropna()
    if df_taxi.empty:
        continue
    
    for row in df_taxi.itertuples():
        gps_time=row[3]
        mins=gps_time.hour*60+gps_time.minute
        
        flow_matrix[day-2][mins//15][int(row[8])]+=1

dyna_id_counter=0   
for day in range(flow_matrix.shape[0]):
    for interval in range(flow_matrix.shape[1]):
        for road in range(flow_matrix.shape[2]):
            write_length=dyna_file.write(f"{dyna_id_counter},state,2019-12-0{day+2}T{str(interval*15//60).zfill(2)}:{str((interval%4)*15).zfill(2)}:00Z,{road},{flow_matrix[day][interval][road]}\n")
            dyna_id_counter+=1
        
dyna_file.close()

 50%|█████     | 61609/123193 [18:34<18:34, 55.27it/s]  


In [3]:
# Mon. to Fri. 12.02-12.04
# 15 min agg interval 24*60/15=96

dyna_file=open("../data/libcity_atom/sz_taxi_3.dyna", "w")
write_length=dyna_file.write("dyna_id,type,time,entity_id,flow\n") # disable printing return value

flow_matrix=np.zeros((3, 96, N), dtype=np.int16)

for taxi_file in tqdm(sorted(os.listdir("../data/taxi_after_proc/tracks"))):
    day=int(taxi_file.split("_")[0].split("-")[1])
    if day>4:
        break
    df_taxi=pd.read_csv(f"../data/taxi_after_proc/tracks/{taxi_file}", parse_dates=["gps_time"]).dropna()
    if df_taxi.empty:
        continue
    
    for row in df_taxi.itertuples():
        gps_time=row[3]
        mins=gps_time.hour*60+gps_time.minute
        
        flow_matrix[day-2][mins//15][int(row[8])]+=1

dyna_id_counter=0   
for day in range(flow_matrix.shape[0]):
    for interval in range(flow_matrix.shape[1]):
        for road in range(flow_matrix.shape[2]):
            write_length=dyna_file.write(f"{dyna_id_counter},state,2019-12-0{day+2}T{str(interval*15//60).zfill(2)}:{str((interval%4)*15).zfill(2)}:00Z,{road},{flow_matrix[day][interval][road]}\n")
            dyna_id_counter+=1
        
dyna_file.close()

 50%|█████     | 61609/123193 [05:11<05:11, 197.79it/s]


In [2]:
# df_dyna.to_csv("../data/libcity_atom/sz_taxi_train.dyna", index=False)
df_dyna=pd.read_csv("../data/libcity_atom/sz_taxi.dyna", nrows=100)

df_dyna

,dyna_id,type,time,entity_id,flow
0,0,state,2019-12-02T00:00:00Z,0,5
1,1,state,2019-12-02T00:00:00Z,1,2
2,2,state,2019-12-02T00:00:00Z,2,9
3,3,state,2019-12-02T00:00:00Z,3,2
4,4,state,2019-12-02T00:00:00Z,4,3
...,...,...,...,...,...
95,95,state,2019-12-02T00:00:00Z,95,0
96,96,state,2019-12-02T00:00:00Z,96,0
97,97,state,2019-12-02T00:00:00Z,97,0
98,98,state,2019-12-02T00:00:00Z,98,1


usr

---

In [56]:
usr_df=pd.DataFrame()
usr_df["usr_id"]=taxi_dict.values()
usr_df["taxi_license"]=taxi_dict.keys()
usr_df

,usr_id,taxi_license
0,0,粤BDD5290
1,1,粤BDK3582
2,2,粤BD04241
3,3,粤BD03542
4,4,粤BD45539
...,...,...
20863,20863,粤BDA5623
20864,20864,粤BD68718
20865,20865,粤BD95572
20866,20866,粤BDD7677


In [60]:
# usr_df.to_csv("../data/libcity_atom/sz_taxi.usr", index=False)

config

---

In [4]:
# config={}

# config["geo"]={}
# config["geo"]["including_types"]=["LineString"]
# config["geo"]["LineString"]={}

# config["rel"]={}
# config["rel"]["including_types"]=["geo"]
# config["rel"]["geo"]={}

# config["usr"]={}
# config["usr"]["properties"]={"taxi_license": "str"}

# config["dyna"]={}
# config["dyna"]["including_types"]=["state"]
# config["dyna"]["state"]={"entity_id": "usr_id", "traj_id": "num", "location": "geo_id", "coordinates": "coordinate", "speed": "num"}

# config["info"]={}
# config["info"]["data_files"]="sz_taxi"
# config["info"]["geo_file"]="sz_taxi"
# config["info"]["rel_file"]="sz_taxi"
# config["info"]["data_col"]=["flow"]
# config["info"]["output_dim"]=1
# # config["info"]["weight_col"]="link_weight"
# # config["info"]["time_intervals"]=60*15
# config["info"]["init_weight_inf_or_zero"]="zero" # adj matrix not connected: 0 (inf: infinity)
# config["info"]["set_weight_link_or_dist"]="link" # adj matrix 01 (dist: use weight)
# config["info"]["calculate_weight_adj"]=False
# # config["info"]["weight_adj_epsilon"]=0.1 # disabled when the above is false

# json.dump(config, open("../data/libcity_atom/config.json", "w", encoding="utf-8"), ensure_ascii=False)
# config

{'geo': {'including_types': ['LineString'], 'LineString': {}},
 'rel': {'including_types': ['geo']},
 'usr': {'properties': {'taxi_license': 'str'}},
 'dyna': {'including_types': ['state'],
  'state': {'entity_id': 'usr_id',
   'traj_id': 'num',
   'location': 'geo_id',
   'coordinates': 'coordinate',
   'speed': 'num'}},
 'info': {'data_files': 'sz_taxi',
  'geo_file': 'sz_taxi',
  'rel_file': 'sz_taxi',
  'data_col': ['speed'],
  'output_dim': 1,
  'init_weight_inf_or_zero': 'zero',
  'set_weight_link_or_dist': 'link',
  'calculate_weight_adj': False}}

In [2]:
import json

config={}

config["geo"]={}
config["geo"]["including_types"]=["LineString"]
config["geo"]["LineString"]={}

config["rel"]={}
config["rel"]["including_types"]=["geo"]
config["rel"]["geo"]={}

config["usr"]={}
config["usr"]["properties"]={"taxi_license": "str"}

config["dyna"]={}
config["dyna"]["including_types"]=["state"]
config["dyna"]["state"]={"entity_id": "geo_id", "flow": "num"}

config["info"]={}
config["info"]["data_files"]="sz_taxi_3"
config["info"]["geo_file"]="sz_taxi"
config["info"]["rel_file"]="sz_taxi"
config["info"]["data_col"]=["flow"]
config["info"]["output_dim"]=1
# config["info"]["weight_col"]="link_weight"
config["info"]["time_intervals"]=60*15
config["info"]["init_weight_inf_or_zero"]="zero" # adj matrix not connected: 0 (inf: infinity)
config["info"]["set_weight_link_or_dist"]="link" # adj matrix 01 (dist: use weight)
config["info"]["calculate_weight_adj"]=False
# config["info"]["weight_adj_epsilon"]=0.1 # disabled when the above is false

json.dump(config, open("../data/libcity_atom/config.json", "w", encoding="utf-8"), ensure_ascii=False)
config

{'geo': {'including_types': ['LineString'], 'LineString': {}},
 'rel': {'including_types': ['geo'], 'geo': {}},
 'usr': {'properties': {'taxi_license': 'str'}},
 'dyna': {'including_types': ['state'],
  'state': {'entity_id': 'geo_id', 'flow': 'num'}},
 'info': {'data_files': 'sz_taxi_3',
  'geo_file': 'sz_taxi',
  'rel_file': 'sz_taxi',
  'data_col': ['flow'],
  'output_dim': 1,
  'time_intervals': 900,
  'init_weight_inf_or_zero': 'zero',
  'set_weight_link_or_dist': 'link',
  'calculate_weight_adj': False}}

In [ ]:
# nohup python run_model.py --task traffic_state_pred --model GRU --dataset sz_taxi --batch_size 5 --gpu_id 5 > flow_GRU.log 2>&1 &